<!--BOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="img/cover-small.jpg" />

Dieses Notizbuch enthält einen angepassten Auszug aus der [Whirlwind Tour of Python](http://www.oreilly.com/programming/free/a-whirlwind-tour-of-python.csp) von Jake VanderPlas; Der Inhalt ist auf [GitHub](https://github.com/jakevdp/WhirlwindTourOfPython) verfügbar.

Text und Code werden unter der [CC0](https://github.com/jakevdp/WhirlwindTourOfPython/blob/master/LICENSE)- Lizenz veröffentlicht; Das Begleitprojekt, das [Python Data Science Handbook](https://github.com/jakevdp/PythonDataScienceHandbook) wird sehr empfohlen.


# Generatoren

Hier werden wir tiefer in die Python-Generatoren eintauchen, einschließlich *Generator-Ausdrücke* und *Generator-Funktionen*.

## Generator-Ausdrücke

Der Unterschied zwischen List Comprehensions und Generator-Ausdrücken ist manchmal verwirrend; im Folgenden werden wir die Unterschiede zwischen ihnen kurz erläutern:

### List comprehensions verwenden eckige Klammern, während Generator-Ausdrücke in Klammern gesetzt werden
Dies ist eine typische List Comprehension:

In [1]:
[n ** 2 for n in range(12)]

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121]

Und das ist ein typischer Generator-Ausdruck:

In [2]:
(n ** 2 for n in range(12))

<generator object <genexpr> at 0x104a60518>

Notice that printing the generator expression does not print the contents; one way to print the contents of a generator expression is to pass it to the ``list`` constructor:

In [3]:
G = (n ** 2 for n in range(12))
list(G)

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121]

### Eine Liste ist eine Sammlung von Werten, während ein Generator ein Rezept zur Erzeugung von Werten ist
Wenn wir eine Liste erstellen, bauen wir eine Sammlung von Werten auf, und das ist mit einigen Speicherkosten verbunden.
Wenn wir einen Generator erstellen, bauen wir keine Wertesammlung auf, sondern ein Rezept zur Erzeugung dieser Werte.
Beide stellen die gleiche Iteratorschnittstelle zur Verfügung, wie hier zu sehen ist:

In [1]:
L = [n ** 2 for n in range(12)]
for val in L:
    print(val, end=' ')

0 1 4 9 16 25 36 49 64 81 100 121 

In [2]:
G = (n ** 2 for n in range(12))
for val in G:
    print(val, end=' ')

0 1 4 9 16 25 36 49 64 81 100 121 

Der Unterschied besteht darin, dass ein Generatorausdruck die Werte erst dann berechnet, wenn sie benötigt werden.
Dies führt nicht nur zu Speichereffizienz, sondern auch zu Recheneffizienz!
Das bedeutet auch, dass die Größe einer Liste durch den verfügbaren Speicher begrenzt ist, während die Größe eines Generatorausdrucks unbegrenzt ist!

Ein Beispiel für einen unendlichen Generatorausdruck kann mit dem in ``itertools`` definierten Iterator ``count`` erstellt werden:

In [6]:
from itertools import count
count()

count(0)

In [7]:
for i in count():
    print(i, end=' ')
    if i >= 10: break

0 1 2 3 4 5 6 7 8 9 10 

Der Iterator ``count`` zählt fröhlich weiter, bis Sie ihm sagen, dass er aufhören soll; das macht es bequem, Generatoren zu erzeugen, die ebenfalls ewig weiterzählen:

In [8]:
factors = [2, 3, 5, 7]
G = (i for i in count() if all(i % n > 0 for n in factors))
for val in G:
    print(val, end=' ')
    if val > 40: break

1 11 13 17 19 23 29 31 37 41 

Sie sehen vielleicht, worauf wir hier hinauswollen: Wenn wir die Liste der Faktoren entsprechend erweitern würden, hätten wir die Anfänge eines Primzahlgenerators, der den Algorithmus des Siebs von Eratosthenes verwendet. Wir werden das gleich noch genauer untersuchen.

### Eine Liste kann mehrfach iteriert werden; ein Generatorausdruck ist nur einmal verwendbar
Dies ist einer der möglichen Nachteile von Generatorausdrücken.
Mit einer Liste können wir dies ganz einfach tun:

In [9]:
L = [n ** 2 for n in range(12)]
for val in L:
    print(val, end=' ')
print()

for val in L:
    print(val, end=' ')

0 1 4 9 16 25 36 49 64 81 100 121 
0 1 4 9 16 25 36 49 64 81 100 121 

Ein Generatorausdruck hingegen ist nach einer Iteration aufgebraucht:

In [10]:
G = (n ** 2 for n in range(12))
list(G)

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121]

In [11]:
list(G)

[]

Dies kann sehr nützlich sein, da es bedeutet, dass die Iteration angehalten und gestartet werden kann:

In [12]:
G = (n**2 for n in range(12))
for n in G:
    print(n, end=' ')
    if n > 30: break

print("\ndoing something in between")

for n in G:
    print(n, end=' ')

0 1 4 9 16 25 36 
doing something in between
49 64 81 100 121 

Dies hat sich als nützlich erwiesen, wenn man mit Sammlungen von Dateien auf der Festplatte arbeitet; so kann man sie ganz einfach in Stapeln analysieren, wobei der Generator den Überblick darüber behält, welche Dateien man noch nicht gesehen hat.

## Generator-Funktionen: Verwendung von ``yield``
Wir haben im vorigen Abschnitt gesehen, dass List Comprehensions am besten dazu geeignet sind, relativ einfache Listen zu erstellen, während die Verwendung einer normalen ``for``-Schleife in komplizierteren Situationen besser ist.
Dasselbe gilt für Generatorausdrücke: Wir können kompliziertere Generatoren mit Hilfe von *Generator-Funktionen* erstellen, die die ``yield``-Anweisung verwenden.

Hier haben wir zwei Möglichkeiten, dieselbe Liste zu konstruieren:

In [13]:
L1 = [n ** 2 for n in range(12)]

L2 = []
for n in range(12):
    L2.append(n ** 2)

print(L1)
print(L2)

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121]
[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121]


Entsprechend haben wir hier zwei Möglichkeiten, gleichwertige Generatoren zu konstruieren:

In [14]:
G1 = (n ** 2 for n in range(12))

def gen():
    for n in range(12):
        yield n ** 2

G2 = gen()
print(*G1)
print(*G2)

0 1 4 9 16 25 36 49 64 81 100 121
0 1 4 9 16 25 36 49 64 81 100 121


Eine Generator-Funktion ist eine Funktion, die anstelle von ``return`` zur einmaligen Rückgabe eines Wertes ``yield`` verwendet, um eine (möglicherweise unendliche) Folge von Werten zu liefern.
Genau wie bei Generatorausdrücken bleibt der Zustand des Generators zwischen den Teiliterationen erhalten, aber wenn wir eine neue Kopie des Generators benötigen, können wir die Funktion einfach erneut aufrufen.

## Beispiel: Primzahlgenerator
Hier folgt eine Funktion, die eine unendliche Reihe von Primzahlen erzeugt, ein Lieblingsbeispiel für eine Generatorfunktion.
Ein klassischer Algorithmus dafür ist das *Sieb des Eratosthenes*, das so funktioniert:

In [15]:
# Generate a list of candidates
L = [n for n in range(2, 40)]
print(L)

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]


In [16]:
# Remove all multiples of the first value
L = [n for n in L if n == L[0] or n % L[0] > 0]
print(L)

[2, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39]


In [17]:
# Remove all multiples of the second value
L = [n for n in L if n == L[1] or n % L[1] > 0]
print(L)

[2, 3, 5, 7, 11, 13, 17, 19, 23, 25, 29, 31, 35, 37]


In [18]:
# Remove all multiples of the third value
L = [n for n in L if n == L[2] or n % L[2] > 0]
print(L)

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37]


Wenn wir diese Prozedur oft genug für eine ausreichend große Liste wiederholen, können wir so viele Primzahlen erzeugen, wie wir wollen.

Wir wollen diese Logik in einer Generatorfunktion kapseln:

In [19]:
def gen_primes(N):
    """Generate primes up to N"""
    primes = set()
    for n in range(2, N):
        if all(n % p > 0 for p in primes):
            primes.add(n)
            yield n

print(*gen_primes(100))

2 3 5 7 11 13 17 19 23 29 31 37 41 43 47 53 59 61 67 71 73 79 83 89 97


Das ist schon alles!
Dies ist sicherlich nicht die rechnerisch effizienteste Implementierung des Siebs des Eratosthenes, aber es zeigt, wie praktisch die Syntax der Generatorfunktion für die Erstellung komplizierterer Sequenzen sein kann.